In [1]:
import cv2
import face_recognition as fr
import numpy as np
import os
from datetime import datetime

In [2]:
path = "faces"
images = []
names = []
lists = os.listdir(path)

In [3]:
for img in lists:
    Img = cv2.imread(f"{path}/{img}")
    images.append(Img)
    names.append(os.path.splitext(img)[0])
print(names)

['Berk', 'Hazal']


In [4]:
def encoding(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = fr.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

In [5]:
def checkAttendance(name):
    with open("attendance.csv","r+") as f:
        data = f.readlines()
        namelist = []
        for line in data:
            entry = line.split(",")
            namelist.append(entry[0])
        if name not in namelist:
            time = datetime.now()
            dstring = time.strftime("%H:%M:%S")
            f.writelines(f"\n{name},{dstring}")


In [6]:
checkAttendance("asd")

In [7]:
encodedList = encoding(images)
print("Encoding is done")

Encoding is done


In [8]:
len(encodedList)

2

In [9]:
capture = cv2.VideoCapture(0)

In [10]:
while True:
    succ, img = capture.read()
    imgSm =cv2.resize(img,(0,0),None,0.25,0.25)
    imgSm = cv2.cvtColor(imgSm, cv2.COLOR_BGR2RGB)

    facesFrame = fr.face_locations(imgSm)
    encodeFrame = fr.face_encodings(imgSm,facesFrame)

    for enc,loc in zip(encodeFrame,facesFrame):
        matching = fr.compare_faces(encodedList,enc)
        distance = fr.face_distance(encodedList,enc)
        #print(distance)
        indexs = np.argmin(distance)

        if matching[indexs]:
            nm = names[indexs].upper()
            #print(nm)
            y1, x2, y2, x1 = loc
            y1, x2, y2, x1 = y1*4, x2*4, y2*4, x1*4
            cv2.rectangle(img, (x1,y1),(x2,y2),(0,255,0),2)
            cv2.rectangle(img, (x1,y2+35),(x2,y2),(0,255,0),cv2.FILLED)
            cv2.putText(img, nm,(x1,y2+25), cv2.FONT_HERSHEY_COMPLEX,1,(255,255,255),2)
            checkAttendance(nm)



    cv2.imshow('Camera',img)
    cv2.waitKey(1)